<a href="https://colab.research.google.com/github/AK-T2024/AK-T2024/blob/main/bert_base_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [2]:
#import python packages

import zipfile
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import seaborn as sns
import pandas as pd

from google.colab import files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Importing the data**

Two Kaggle datasets are required.

1. Hate Speech and Offensive language dataset: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset

2. Twitter Sentiment Analysis dataset: https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset

In [3]:
# import kaggle data:


# Install Kaggle API and authenticate
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Verify Kaggle API
!kaggle datasets list

# Hate Speech Dataset
# --------------------

# Download the hate speech dataset
!kaggle datasets download -d mrmorj/hate-speech-and-offensive-language-dataset

# Unzip the hate speech dataset
if not os.path.exists('hate-speech-dataset'):
    os.makedirs('hate-speech-dataset')

with zipfile.ZipFile('hate-speech-and-offensive-language-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('hate-speech-dataset')

# Load the dataset into a DataFrame
dataset_path = 'hate-speech-dataset/labeled_data.csv'
df_hate = pd.read_csv(dataset_path)

# Display the first few rows of the DataFrame
print(df_hate.head())

# Sentiment Analysis Dataset
# --------------------------

# Download the dataset
!kaggle datasets download -d yasserh/twitter-tweets-sentiment-dataset

# Unzip the dataset
if not os.path.exists('twitter-sentiment-dataset'):
    os.makedirs('twitter-sentiment-dataset')

with zipfile.ZipFile('twitter-tweets-sentiment-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('twitter-tweets-sentiment-dataset')

# Load the dataset into a DataFrame
dataset_path = 'twitter-tweets-sentiment-dataset/Tweets.csv'
df_neutral = pd.read_csv(dataset_path)

# Display the first few rows of the DataFrame
print(df_neutral.head())

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Dataset URL: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset
License(s): CC0-1.0
 99% 1.00M/1.01M [00:00<00:00, 1.42MB/s]
100% 1.01M/1.01M [00:00<00:00, 1.42MB/s]
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0    

In [4]:
# Drop columns except class and tweet:
df_hate = df_hate.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'], axis=1)

# Retain only the 'hate_speech' (class = 0) and 'offensive_language' (class = 1) tweets:
df_hate = df_hate.loc[df_hate['class'].isin([0, 1]), ['class', 'tweet']]

# Set the 'class' to be 0 (hate)
df_hate['class'] = 0

In [5]:
# Retain only rows where 'sentiment' is 'positive' or 'neutral'
df_neutral = df_neutral.loc[df_neutral['sentiment'].isin(['positive', 'neutral']), ['text', 'sentiment']].copy()

# Set the 'class' of tweet to be 1
df_neutral['class'] = 1

# Retain only the 'text' and 'class' columns
df_neutral = df_neutral[['class', 'text']]

# Rename the 'text' column as 'tweet'
df_neutral.rename(columns={'text': 'tweet'}, inplace=True)

In [6]:
# Concatinate
df= pd.concat([df_hate, df_neutral], ignore_index=True)
df.head()

,class,tweet
0,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
1,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
2,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
3,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
4,0,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."


In [9]:
# Function: clean data

def clean_text(text, additional_words=None):
    if isinstance(text, str):
        # Convert to lowercase
        #text = text.lower()

        # Remove URLs and Twitter handles
       # text = re.sub(r'http\S+|www\S+|https\S+|@[\w_]+', '', text)

        # Remove special characters and numerical digits
        #text = re.sub(r'[^a-z\s]', '', text)

        # Get the list of stop words and merge with additional words if provided
        #stop_words = set(stopwords.words("english"))
        #if additional_words:
        #    stop_words.update(additional_words)

        # Remove stop words and additional words
        #words = [word for word in text.split() if word not in stop_words]

        # Join the words back into a string
        #cleaned_text = " ".join(words).strip()  # Strip leading/trailing spaces
        cleaned_text = text.lower()

        return cleaned_text if cleaned_text else None  # Return None for empty strings
    else:
        return None  # Return None for non-string values

In [10]:
# Set additional words to be removed - these are extra to the nltk list of stop words
additional_words = ['rt', 'amp', 'really', 'look', 'like', 'want', 'one', 'two']

# Apply the clean_text function:
additional_words = ['rt', 'amp', 'really', 'look', 'like', 'want', 'one', 'two']

df['clean_tweet'] = df['tweet'].apply(clean_text, additional_words=additional_words)
df.head()

,class,tweet,clean_tweet
0,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,!!!!! rt @mleew17: boy dats cold...tyga dwn ba...
1,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,!!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...
2,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,!!!!!!!!! rt @c_g_anderson: @viva_based she lo...
3,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,!!!!!!!!!!!!! rt @shenikaroberts: the shit you...
4,0,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...","!!!!!!!!!!!!!!!!!!""@t_madison_x: the shit just..."


In [12]:
# Remove rows where 'clean_tweet' is None
df= df[df['clean_tweet'].notna()]

# Reset the index
df.reset_index(drop=True, inplace=True)

In [13]:
# Features and target
X = df['clean_tweet']
y = df['class']

# Split the data into training (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Further split the temp data into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Create DataFrames for each subset
train_df = pd.DataFrame({'clean_tweet': X_train, 'class': y_train})
val_df = pd.DataFrame({'clean_tweet': X_val, 'class': y_val})
test_df = pd.DataFrame({'clean_tweet': X_test, 'class': y_test})

print(len(train_df), len(val_df), len(test_df))

24191 8064 8064


**Bert Model**

In [14]:
from transformers import TFAutoModel
from google.colab import userdata
my_secret_key = userdata.get('WAI_token')

In [15]:
model2 = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
import tensorflow as tf

class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [18]:
classifier = BERTForClassification(model2, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [19]:
# Tokenization function
def tokenize(texts, labels, max_length=128):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length)
    return tf.data.Dataset.from_tensor_slices((dict(encodings), labels))

# Convert pandas DataFrames to TensorFlow datasets
train_texts, train_labels = train_df['clean_tweet'], train_df['class']
val_texts, val_labels = val_df['clean_tweet'], val_df['class']
test_texts, test_labels = test_df['clean_tweet'], test_df['class']

# Create TensorFlow datasets
train_dataset = tokenize(train_texts, train_labels)
val_dataset = tokenize(val_texts, val_labels)
test_dataset = tokenize(test_texts, test_labels)

# Batch and shuffle the datasets
BATCH_SIZE = 8
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Define the BERT-based classification model
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        x = self.bert(input_ids, attention_mask=attention_mask)[1]
        return self.fc(x)

# Initialize the classifier
num_classes = 2
classifier = BERTForClassification(model2, num_classes)

# Compile the classifier
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Train the model
EPOCHS = 3
classifier.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

# Evaluate the model on the test dataset
loss, accuracy = classifier.evaluate(test_dataset)
print(f'Test Accuracy: {accuracy}')

Epoch 1/3
3024/3024 [==============================] - 788s 246ms/step - loss: 0.0374 - accuracy: 0.9874 - val_loss: 0.0151 - val_accuracy: 0.9958
Epoch 2/3
3024/3024 [==============================] - 740s 245ms/step - loss: 0.0073 - accuracy: 0.9977 - val_loss: 0.0197 - val_accuracy: 0.9955
Epoch 3/3
1008/1008 [==============================] - 83s 82ms/step - loss: 0.0156 - accuracy: 0.9969
Test Accuracy: 0.9968997836112976


In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

# Obtain predictions from the model
y_pred_probs = classifier.predict(test_dataset)

# Convert probabilities to class predictions (assuming binary classification for simplicity)
y_pred = np.argmax(y_pred_probs, axis=1)


# Extract true labels from the test_dataset
y_true = []
for batch in test_dataset:
    # Assuming labels are the second element in the tuple
    y_true.extend(batch[1].numpy())

# Convert y_true to numpy array
y_true = np.array(y_true)

# Calculate the precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the metrics
print(f'Test Precision: {precision}')
print(f'Test Recall: {recall}')
print(f'Test F1 Score: {f1}')

# Optionally, print a detailed classification report
print(classification_report(y_true, y_pred))

1008/1008 [==============================] - 86s 82ms/step
Test Precision: 0.9969004783884056
Test Recall: 0.9968998015873016
Test F1 Score: 0.9968997565098984
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4124
           1       1.00      1.00      1.00      3940

    accuracy                           1.00      8064
   macro avg       1.00      1.00      1.00      8064
weighted avg       1.00      1.00      1.00      8064



In [ ]:
from google.colab import userdata
userdata.get('WAI_token')

In [23]:
from datasets import load_dataset
# Load the 'ethos' dataset with the 'binary' variant for the 'train' split
dataset = load_dataset('ethos', 'binary', split='train')
# Display the loaded dataset
print(dataset)

The repository for ethos contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ethos.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/998 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 998
})


In [24]:
# Convert the dataset to a pandas DataFrame
df_test = dataset.to_pandas()

df_test['clean_tweet'] = df_test['text'].apply(clean_text, additional_words=additional_words)
df_test.head()

,text,label,clean_tweet
0,You should know women's sports are a joke,1,you should know women's sports are a joke
1,You look like Sloth with deeper Down’s syndrome,1,you look like sloth with deeper down’s syndrome
2,You look like Russian and speak like Indian. B...,1,you look like russian and speak like indian. b...
3,"Women deserve to be abused, I guess.",1,"women deserve to be abused, i guess."
4,Women are made for making babies and cooking d...,1,women are made for making babies and cooking d...


In [25]:
# Define the mapping
class_mapping = {0: 1, 1: 0}

# Apply the mapping to the 'class' column
df_test['class'] = df_test['label'].map(class_mapping)
df_test.head()

,text,label,clean_tweet,class
0,You should know women's sports are a joke,1,you should know women's sports are a joke,0
1,You look like Sloth with deeper Down’s syndrome,1,you look like sloth with deeper down’s syndrome,0
2,You look like Russian and speak like Indian. B...,1,you look like russian and speak like indian. b...,0
3,"Women deserve to be abused, I guess.",1,"women deserve to be abused, i guess.",0
4,Women are made for making babies and cooking d...,1,women are made for making babies and cooking d...,0


In [26]:
test_texts, test_labels = df_test['clean_tweet'], df_test['class']
test_dataset = tokenize(test_texts, test_labels)

test_dataset = test_dataset.batch(BATCH_SIZE)
loss, accuracy = classifier.evaluate(test_dataset)
print(f'Test Accuracy: {accuracy}')



125/125 [==============================] - 10s 83ms/step - loss: 1.8348 - accuracy: 0.6844
Test Accuracy: 0.6843687295913696


In [27]:
# Obtain predictions from the model
y_pred_probs = classifier.predict(test_dataset)

# Convert probabilities to class predictions (assuming binary classification for simplicity)
y_pred = np.argmax(y_pred_probs, axis=1)


# Extract true labels from the test_dataset
y_true = []
for batch in test_dataset:
    # Assuming labels are the second element in the tuple
    y_true.extend(batch[1].numpy())

# Convert y_true to numpy array
y_true = np.array(y_true)

# Calculate the precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the metrics
print(f'Test Precision: {precision}')
print(f'Test Recall: {recall}')
print(f'Test F1 Score: {f1}')

# Optionally, print a detailed classification report
print(classification_report(y_true, y_pred))


125/125 [==============================] - 10s 82ms/step
Test Precision: 0.6818579358724319
Test Recall: 0.6843687374749499
Test F1 Score: 0.6804929539288851
              precision    recall  f1-score   support

           0       0.66      0.57      0.61       433
           1       0.70      0.78      0.74       565

    accuracy                           0.68       998
   macro avg       0.68      0.67      0.67       998
weighted avg       0.68      0.68      0.68       998

